#### Universidad Autónoma de Aguascalientes
#### Departamento: Ciencias de la Computación
#### Carrera: Ingenieria en Computación Inteligente
#### Curso: Machine Learning y Deep Learning 
#### Maestro: Dr. Francisco Javier Luna Rosas

#### Alumno: Jorge Hernández
#### Semestre: Enero - Junio del 2025

##### PRACTICA No. _. Red Neuronal BackPropagation
En esta práctica se implementará un modelo de clasificación una red neuronal con la técnica de propagación hacia atras (Por sus siglas en el idioma inglés NNBP). 

<p align="center">
  <img width="460" height="300" src="../Practica_3_Red%20BackPropagation/assets/red_Backpr.png">
</p>

En la figura se muestra una red neuronal de avance multicapa con propagación hacia adelante, siendo la tasa de aprendizaje de 0.9. 

Los valores inciales de peso y sesgo (bias) se dan en la tabla a continuación, junto con la primera tupla de entrenamiento, X = (1,0,1), cuaya etiqueta es de clase 1.

$$
\begin{array}{|c|c|c|c|c|c|c|c|c|c|c|c|c|c|}
\hline
x_1 & x_2 & x_3 & w_{14} & w_{15} & w_{24} & w_{25} & w_{34} & w_{35} & w_{46} & w_{56} & \theta_4 & \theta_5 & \theta_6 \\ \hline
1 & 0 & 1 & 0.2 & -0.3 & 0.4 & 0.1 & -0.5 & 0.2 & -0.3 & -0.2 & -0.4 & 0.2 & 0.1 \\ \hline
\end{array}
$$


#### CODIGO

In [32]:
import numpy as np

In [33]:
class RedNeuronalBackPropagation:
    def __init__(self,tazadeAprendizaje, epocas):
        print("AJUSTANDO PARAMETRO INCIALES")
        self.tazadeAprendizaje = tazadeAprendizaje
        self.capa_entrada = 3
        self.capa_oculta = 2
        self.capa_salida = 1
        self.bandera = False
        self.epocas = epocas
        
        self.weights = {
            (1,4): 0.2,
            (1,5): -0.3,
            (2,4): 0.4,
            (2,5): 0.1,
            (3,4): -0.5,
            (3,5): 0.2,
            (4,6): -0.3,
            (5,6): -0.2
        }

        self.bias = {
            4 : -0.4,
            5 : 0.2,
            6 : 0.1
        }
        pass

    def funcion_sigmoidal(self,x):
        return 1/(1 + np.exp(-x))
    
    def red_backpr(self,values_w,tupla_entre,y,x):
        index_capa_salida = 6

        for capa_out in range(self.capa_salida):
            self.values_w[("er",index_capa_salida)] = (values_w[("o",index_capa_salida)] * (1 - values_w[("o",index_capa_salida)]) * (y[tupla_entre][0]- values_w[("o",index_capa_salida)]))

        for capa_hidden in range(self.capa_oculta):
            self.values_w[("er",capa_hidden+self.capa_entrada+1)] = (values_w[("o",capa_hidden+self.capa_entrada+1)] * (1-values_w[("o",capa_hidden+self.capa_entrada+1)]) * (self.values_w[("er",index_capa_salida)]) * (self.weights[(capa_hidden+self.capa_entrada+1, index_capa_salida)]))
            
        for index_input in range(self.capa_entrada):
            for index_hidden in range(self.capa_oculta):
                input_hidden_key = (index_input + 1, index_hidden + self.capa_entrada + 1)
                error_key = ("er", index_hidden + self.capa_entrada + 1)

                current_weight = self.weights[input_hidden_key]
                error_value = self.values_w[error_key]
                input_value = x[tupla_entre][index_input]

                self.weights[input_hidden_key] = current_weight + (self.tazadeAprendizaje * error_value * input_value)
        
        for index_hidden in range(self.capa_oculta):
            for index_output in range(self.capa_salida):
                hidden_output_key = (index_hidden + self.capa_entrada + 1, index_output + self.capa_entrada + self.capa_oculta + 1)
                error_key = ("er", index_hidden+1+self.capa_entrada)

                current_weight = self.weights[hidden_output_key]
                error_value = self.values_w[("er",index_output + self.capa_entrada + self.capa_oculta + 1)]
                hidden_value = self.values_w[("o",self.capa_entrada+1+index_hidden)]

                self.weights[hidden_output_key] = current_weight + (self.tazadeAprendizaje * error_value * hidden_value)

        for index in self.bias:
            self.bias[(index)] = self.bias[(index)]+((self.tazadeAprendizaje)*(self.values_w[("er",index)]))

    def red_forward(self,x,y,values_w):
        #for _ in range(self.epocas):
            # for i in range(len(x)):
            #     for capa_i in range(self.capa_entrada):
            #         self.values_w[("i",capa_i)] = x[i][capa_i]
                for neuron_o in range(self.capa_oculta):
                    net_input = sum(
                        self.weights[(neuron_i + 1, neuron_o + self.capa_entrada + 1)] * self.values_w[("i", neuron_i)]
                        for neuron_i in range(self.capa_entrada)
                    )
                    net_input += self.bias[neuron_o + self.capa_entrada + 1]
                    self.values_w[("o", neuron_o + self.capa_entrada + 1)] = self.funcion_sigmoidal(net_input)


                for neuron_s in range(self.capa_salida):
                    net_input = sum(
                        self.weights[(neuron_h + self.capa_entrada + 1, neuron_s + self.capa_entrada + self.capa_oculta + 1)] 
                        * self.values_w[("o", neuron_h + self.capa_entrada + 1)]
                        for neuron_h in range(self.capa_oculta)
                    )

                    net_input += self.bias[neuron_s + self.capa_entrada + self.capa_oculta + 1]
                    self.values_w[("o", neuron_s + self.capa_entrada + self.capa_oculta + 1)] = self.funcion_sigmoidal(net_input)
                
    def train(self,x,y):
        self.values_w = {}

        for inicializer in range(self.capa_oculta+self.capa_salida):
            self.values_w["o",inicializer+self.capa_entrada+1] = 0
            self.values_w["er",inicializer+self.capa_entrada+1] = 0

        for _index in range(self.epocas):
            for i in range(len(x)):

                for capa_i in range(self.capa_entrada):
                    self.values_w[("i",capa_i)] = x[i][capa_i]

            self.red_forward(x,y,self.values_w)
            self.red_backpr(self.values_w, i, y, x)

            print(f"Epoca {_index+1}: {round(self.values_w[("o"), 6], 5)}")

        print("RED ENTRENADA")
        self.bandera = True

In [34]:
x = np.array([[1,0,1]])
y = np.array([[1]])

#x = np.array([[1,0,1],[1,0,1]])
#y = np.array([[1],[1]])

red_nb = RedNeuronalBackPropagation(tazadeAprendizaje=0.9, epocas=500)
red_nb.train(x,y)
#red_nb.red_forward(x,y)

AJUSTANDO PARAMETRO INCIALES
Epoca 1: 0.47389
Epoca 2: 0.51525
Epoca 3: 0.55287
Epoca 4: 0.58648
Epoca 5: 0.61618
Epoca 6: 0.64228
Epoca 7: 0.66519
Epoca 8: 0.68534
Epoca 9: 0.70311
Epoca 10: 0.71885
Epoca 11: 0.73285
Epoca 12: 0.74538
Epoca 13: 0.75663
Epoca 14: 0.7668
Epoca 15: 0.77601
Epoca 16: 0.7844
Epoca 17: 0.79208
Epoca 18: 0.79912
Epoca 19: 0.80561
Epoca 20: 0.81161
Epoca 21: 0.81717
Epoca 22: 0.82234
Epoca 23: 0.82716
Epoca 24: 0.83166
Epoca 25: 0.83588
Epoca 26: 0.83984
Epoca 27: 0.84357
Epoca 28: 0.84709
Epoca 29: 0.85041
Epoca 30: 0.85356
Epoca 31: 0.85654
Epoca 32: 0.85936
Epoca 33: 0.86205
Epoca 34: 0.86461
Epoca 35: 0.86706
Epoca 36: 0.86939
Epoca 37: 0.87161
Epoca 38: 0.87375
Epoca 39: 0.87579
Epoca 40: 0.87775
Epoca 41: 0.87963
Epoca 42: 0.88143
Epoca 43: 0.88317
Epoca 44: 0.88484
Epoca 45: 0.88645
Epoca 46: 0.888
Epoca 47: 0.8895
Epoca 48: 0.89095
Epoca 49: 0.89234
Epoca 50: 0.8937
Epoca 51: 0.895
Epoca 52: 0.89627
Epoca 53: 0.8975
Epoca 54: 0.89868
Epoca 55: 0.89984